In [ ]:
import gc
import os
import sys
import glob
import numpy as np
import pandas as pd
import scanpy as sc
import scipy.sparse
import scvelo as scv
import anndata as ad
import scipy.io as sio
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
ProgressBar().register()

In [ ]:
def generate_splice_matrices(tscp_path, cutoff, adata_path):
    print(f"Reading in {tscp_path}")
    tscp_assign_df = dd.read_csv(tscp_path, blocksize="800MB")
    
    tscp_assign_df = tscp_assign_df.compute()
    cell_tscp_cnts = tscp_assign_df.groupby("bc_wells").size()
    cell_tscp_cnts = cell_tscp_cnts[cell_tscp_cnts >= cutoff]
    filtered_cell_dict = dict(zip(cell_tscp_cnts.index,np.zeros(len(cell_tscp_cnts))))
    
    def check_filtered_cell(cell_ind):
        try:
            filtered_cell_dict[cell_ind]
        except:
            return False
        else:
            return True
    
    genes = tscp_assign_df.gene_name.unique()
    bcs = cell_tscp_cnts.index
    gene_dict = dict(zip(genes,range(len(genes))))
    barcode_dict = dict(zip(bcs,range(len(bcs))))
    reads_to_keep = tscp_assign_df.bc_wells.apply(check_filtered_cell)
    
    print("\nFiltering tscp file..")
    tscp_assign_df_filt = tscp_assign_df[reads_to_keep]
    tscp_assign_df_filt["cell_index"] = tscp_assign_df_filt.bc_wells.apply(lambda s:barcode_dict[s])
    tscp_assign_df_filt["gene_index"] = tscp_assign_df_filt.gene_name.apply(lambda s:gene_dict[s])
    print("Done:", tscp_assign_df_filt.shape)
    
    rcv = tscp_assign_df_filt.query("exonic").groupby(["cell_index","gene_index"]).size().reset_index().values
    rows = list(rcv[:,0])+[len(barcode_dict)-1]
    cols = list(rcv[:,1])+[len(genes)-1]
    vals = list(rcv[:,2])+[0]
    X_exonic = scipy.sparse.csr_matrix((vals,(rows,cols)))
    
    rcv = tscp_assign_df_filt.query("~exonic").groupby(["cell_index","gene_index"]).size().reset_index().values
    rows = list(rcv[:,0])+[len(barcode_dict)-1]
    cols = list(rcv[:,1])+[len(genes)-1]
    vals = list(rcv[:,2])+[0]
    X_intronic = scipy.sparse.csr_matrix((vals,(rows,cols)))
    
    X = X_exonic + X_intronic
    adata = scv.AnnData(X=X,)
    
    x_row, x_col = adata.shape
    adata.obs = pd.DataFrame({"barcodes":bcs}, index=bcs)
    adata.var = pd.DataFrame({"gene":genes,"gene_name":genes})
    adata.var.index = genes
    
    adata.var_names_make_unique()
    adata.obs_names_make_unique()
    adata.layers["spliced"] = X_exonic
    adata.layers["unspliced"] = X_intronic
    scv.utils.show_proportions(adata)
    
    adata.obs.index = adata.obs.index.astype(str)
    adata.var.index = adata.var.index.astype(str)
    adata.write(f"{adata_path}sub{subs[i]}_adata.h5ad")
    print(f"Saved anndata to {adata_path}{subs[i]}_adata.h5ad.\n")

# Data process

In [ ]:
working_dir = '/workdir/jp2626/chicken/scVelo/'
tscp_paths = glob.glob(f"/workdir/ra574/slideseq/Parse_Chick_Heart/analysis/*/process/tscp_assignment.csv.gz")
tscp_paths_tounzip = [
    path.replace(
        '/workdir/ra574/slideseq/Parse_Chick_Heart/analysis/', 
        '/workdir/jp2626/chicken/scVelo/'
    ) for path in tscp_paths
]
subs = [*range(1,9)]


tscp_unzipped = []
# Unzip tscp file, if not already done
for i in range(len(tscp_paths)):
    original_path = tscp_paths[i]
    target_path = tscp_paths_tounzip[i]
    
    # Create directory if not exists
    os.makedirs(os.path.dirname(target_path), exist_ok=True)
    
    # If unzipped file does not exist
    if not os.path.exists(target_path.replace(".gz", "")):
        # Copy the gz file
        os.system(f"cp {original_path} {target_path}")
        
        # Unzip at the new location
        os.system(f"pigz -k -d {target_path}")
        
        # Remove the copied .gz file
        os.remove(target_path)

    # Append unzipped path
    tscp_unzipped.append(target_path.replace(".gz", ""))

print(tscp_unzipped)

In [ ]:
ad_list_sp = []
for i in range(len(tscp_paths)):
    ad_list_sp.append(generate_splice_matrices(tscp_unzipped[i], tscp_cutoffs[i], f"{working_dir}"))

In [ ]:
tscp_cutoffs = []
tscp_cutoffs_paths = glob.glob(f"/workdir/ra574/slideseq/Parse_Chick_Heart/analysis/202303_ex1_sub*/agg_samp_ana_summary.csv")
for path in tscp_cutoffs_paths:
    tmp = pd.read_csv(path, index_col  = 'statistic')
    tscp_cutoffs.append(tmp.loc['cell_tscp_cutoff', 'all-well'])

for i in range(len(tscp_paths)):
    ad_list_sp.append(generate_splice_matrices(tscp_unzipped[i], tscp_cutoffs[i], f"{working_dir}"))

In [ ]:
ad_paths = glob.glob(f"/workdir/jp2626/chicken/scVelo/sub*.h5ad")
ad_list_sp = []

for ad in ad_paths:
    ad_list_sp.append(sc.read_h5ad(ad))
ad_splice = ad.concat(ad_list_sp, keys=subs,index_unique="__s")

# Subset processed scVelo adata

In [ ]:
# Cardiomyocytes
adata_sc = sc.read_h5ad('./chicken_qc_processed_rohitadd_cardi.h5ad')
inter_index = ad_splice.obs.index.intersection(adata_sc.obs.index)
ad_splice_sub = ad_splice[inter_index].copy()
adata_sc_sub = adata_sc[inter_index].copy()
ad_splice_sub.obs['sub_celltype_v3_fin'] = adata_sc_sub.obs['sub_celltype_v3_fin']
ad_splice_sub.obs['day'] = adata_sc_sub.obs['day']
ad_splice_sub.obs['site'] = adata_sc_sub.obs['site']
ad_splice_sub.obs['global_celltype'] = adata_sc_sub.obs['global_celltype']
ad_splice_sub.obsm['X_pca'] = adata_sc_sub.obsm['X_pca']
ad_splice_sub.obsm['X_umap'] = adata_sc_sub.obsm['X_umap']
ad_splice_sub.write_h5ad('/workdir/jp2626/chicken/scVelo/chicken_qc_processed_cardi_spliced.h5ad')

# Endothelial cells
adata_sc = sc.read_h5ad('./chicken_qc_processed_rohitadd_endo.h5ad')
inter_index = ad_splice.obs.index.intersection(adata_sc.obs.index)
ad_splice_sub = ad_splice[inter_index].copy()
adata_sc_sub = adata_sc[inter_index].copy()
ad_splice_sub.obs['sub_celltype_v3_fin'] = adata_sc_sub.obs['sub_celltype_v3_fin']
ad_splice_sub.obs['day'] = adata_sc_sub.obs['day']
ad_splice_sub.obs['site'] = adata_sc_sub.obs['site']
ad_splice_sub.obs['global_celltype'] = adata_sc_sub.obs['global_celltype']
ad_splice_sub.obsm['X_pca'] = adata_sc_sub.obsm['X_pca']
ad_splice_sub.obsm['X_umap'] = adata_sc_sub.obsm['X_umap']
ad_splice_sub.write_h5ad('/workdir/jp2626/chicken/scVelo/chicken_qc_processed_endo_spliced.h5ad')

# Fibroblasts
adata_sc = sc.read_h5ad('./chicken_qc_processed_rohitadd_fib.h5ad')
inter_index = ad_splice.obs.index.intersection(adata_sc.obs.index)
ad_splice_sub = ad_splice[inter_index].copy()
adata_sc_sub = adata_sc[inter_index].copy()
ad_splice_sub.obs['sub_celltype_v3_fin'] = adata_sc_sub.obs['sub_celltype_v3_fin']
ad_splice_sub.obs['day'] = adata_sc_sub.obs['day']
ad_splice_sub.obs['site'] = adata_sc_sub.obs['site']
ad_splice_sub.obs['global_celltype'] = adata_sc_sub.obs['global_celltype']
ad_splice_sub.obsm['X_pca'] = adata_sc_sub.obsm['X_pca']
ad_splice_sub.obsm['X_umap'] = adata_sc_sub.obsm['X_umap']
ad_splice_sub.write_h5ad('/workdir/jp2626/chicken/scVelo/chicken_qc_processed_fib_spliced.h5ad')

# Epithelial cells
adata_sc = sc.read_h5ad('./chicken_qc_processed_rohitadd_epi.h5ad')
inter_index = ad_splice.obs.index.intersection(adata_sc.obs.index)
ad_splice_sub = ad_splice[inter_index].copy()
adata_sc_sub = adata_sc[inter_index].copy()
ad_splice_sub.obs['sub_celltype_v3_fin'] = adata_sc_sub.obs['sub_celltype_v3_fin']
ad_splice_sub.obs['day'] = adata_sc_sub.obs['day']
ad_splice_sub.obs['site'] = adata_sc_sub.obs['site']
ad_splice_sub.obs['global_celltype'] = adata_sc_sub.obs['global_celltype']
ad_splice_sub.obsm['X_pca'] = adata_sc_sub.obsm['X_pca']
ad_splice_sub.obsm['X_umap'] = adata_sc_sub.obsm['X_umap']
ad_splice_sub.write_h5ad('/workdir/jp2626/chicken/scVelo/chicken_qc_processed_epi_spliced.h5ad')

# Run scVelo + Plasticity score calculation

###### Fibroblasts for example - do the same for cardiomyocytes, endothelial cells, epithelial cells

## scVelo

In [ ]:
ad_splice_sub = sc.read_h5ad('/workdir/jp2626/chicken/scVelo/chicken_qc_processed_fib_spliced.h5ad')
scv.pp.moments(ad_splice_sub, n_pcs=50, n_neighbors=30)
scv.tl.velocity(ad_splice_sub, mode="stochastic")
scv.tl.velocity_graph(ad_splice_sub)
scv.pl.velocity_embedding_stream(ad_splice_sub, basis="umap", color="sub_celltype_v3",size=30 ,alpha=0.8 ,fontsize=10)
scv.tl.velocity_pseudotime(ad_splice_sub)
scv.pl.scatter(ad_splice_sub, color='velocity_pseudotime', cmap='gnuplot')
ad_splice_sub.uns['neighbors']['distances'] = ad_splice_sub.obsp['distances']
ad_splice_sub.uns['neighbors']['connectivities'] = ad_splice_sub.obsp['connectivities']
scv.tl.paga(ad_splice_sub, groups='sub_celltype_v3')
scv.pl.paga(ad_splice_sub, basis='umap', size=50, alpha=.1,
            min_edge_width=2, node_size_scale=1.5)

### Calculate Plasticity score

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Compute Magnitude (Euclidean Norm == L2 norm to know the magnitude of the velocity vector)
velocity_vectors = adata.obsm["velocity_umap"]
magnitude = np.linalg.norm(velocity_vectors, axis=1)  # Higher magnitude: early stage, Lower magnitude: Later stage
magnitude_scaled = scaler.fit_transform(magnitude.reshape(-1, 1)).flatten() # To avoid negative value scale it to 0 to 1
# Compute Net flow (mean Outflow prob - mean Inflow prob)
import numpy as np
T = adata.uns['velocity_graph']  # sparse matrix
# Total outflow/inflow per cell
outflow = np.array(T.sum(axis=1)).flatten()  # Sum of each row (outflow)
inflow = np.array(T.sum(axis=0)).flatten()   # Sum of each column (inflow)
# Count non-zero elements for outflow and inflow (row-wise and column-wise)
non_zero_counts_outflow = np.array(np.sum(T > 0, axis=1)).flatten()  # Non-zero counts for outflow (row-wise)
non_zero_counts_inflow = np.array(np.sum(T > 0, axis=0)).flatten() # Non-zero counts for inflow (column-wise)
# Calculate the mean outflow and inflow excluding zeros
outflow_mean_non_zero = outflow / (non_zero_counts_outflow + 1e-6)  # Add small epsilon to avoid division by zero
inflow_mean_non_zero = inflow / (non_zero_counts_inflow + 1e-6)    # Same here
# Source/sink score: net flow score
net_flow = outflow_mean_non_zero   -  inflow_mean_non_zero  # Higher netflow: Early stage. Lower netflow: Stable stage
net_flow_scaled = scaler.fit_transform(net_flow.reshape(-1, 1)).flatten() # To avoid negative value scale it to 0 to 1
velocity_score =   magnitude_scaled * net_flow_scaled
adata.obs['velocity_score_netflow'] =  velocity_score
adata.write_h5ad('/workdir/jp2626/chicken/scVelo/chicken_qc_processed_fib_spliced.h5ad')
scv.pl.scatter(adata, color='velocity_score_netflow', cmap='gnuplot_r')

# Velocity score per cell type
grouped_means = adata.obs.groupby('sub_celltype_v3_fin')['velocity_score_netflow'].mean().reset_index()
# Invert *before* min-max scaling
grouped_means["velocity_score_netflow"] =  grouped_means["velocity_score_netflow"]
# Now apply min-max scaling: high = later stage
grouped_means["velocity_score_minmax_scaled"] = (
    grouped_means["velocity_score_netflow"] - grouped_means["velocity_score_netflow"].min()
) / (grouped_means["velocity_score_netflow"].max() - grouped_means["velocity_score_netflow"].min())
grouped_means.to_csv('/workdir/jp2626/chicken/scVelo/FB_velocity_netflow_score.csv')

# Map Plasticity score to tissue

In [ ]:
def get_suffix(sample):
    return "S" if "_S" in sample else "L"
adata_files = glob.glob("/fs/cbsuvlaminck5/workdir/jp2626/chickenheart/jy/graphst_res/noery/*_S/*mclust*.h5ad", recursive=True) + glob.glob("/fs/cbsuvlaminck5/workdir/jp2626/chickenheart/jy/graphst_res/noery/*_L/*mclust*.h5ad", recursive=True) 
sample_names = [path.split('/')[9].replace('ST_', '') for path in adata_files]

In [ ]:
grouped_means = pd.read_csv('/workdir/jp2626/chicken/scVelo/FB_velocity_netflow_score.csv')
velocity = dict(zip(
    grouped_means['sub_celltype_v3'],
    grouped_means['velocity_score_minmax_scaled']
))

# Compute max range and centers
max_x_range, max_y_range = 0.0, 0.0
center_points = []

for sample in sample_names:
    suffix = get_suffix(sample)
    adata = sc.read_h5ad(f"/fs/cbsuvlaminck3/workdir/jp2626/chickenheart_cell2loc/{sample}/cell2location_map_v3_cutoff2_0.03_{suffix}/sp.h5ad")
    coords = adata.obsm["spatial"]
    x_max, x_min = coords[:, 0].max(), coords[:, 0].min()
    y_max, y_min = coords[:, 1].max(), coords[:, 1].min()
    max_x_range = max(max_x_range, x_max - x_min)
    max_y_range = max(max_y_range, y_max - y_min)
    center_points.append([(x_max + x_min) / 2, (y_max + y_min) / 2])


# Plotting
fig, axs = plt.subplots(1, len(sample_names), figsize=(24, 6))
maxscale = 0.4
minscale = 0.1

for i, sample in enumerate(sample_names):
    suffix = get_suffix(sample)
    cell2loc = sc.read_h5ad(f'/fs/cbsuvlaminck3/workdir/jp2626/chickenheart_cell2loc/{sample}/cell2location_map_v3_cutoff2_0.03_{suffix}/sp.h5ad')
    cell2loc = cell2loc[cell2loc.obs['max_pred_celltype'] != 'Erythrocytes'].copy()
    score = pd.read_csv(f'/workdir/jp2626/chicken/scVelo/netflow_Stagescore_FB_{sample}.csv' , index_col='Unnamed: 0')
    score_filt = score[score.index.isin(cell2loc.obs.index)]
    score_filt = score_filt.reindex(cell2loc.obs.index)
    cell2loc.obs['Stagescore_FB'] = score_filt['Stagescore_FB']

    
    # Plot
    sc.pl.spatial(cell2loc, color="Stagescore_FB", spot_size=30, cmap='viridis',
                  ax=axs[i], show=False,frameon=False, colorbar_loc=None ,vmax=maxscale, vmin=minscale, title = "")  
    
    # Use center + max range to align and avoid cropping
    cx, cy = center_points[i]
    axs[i].set_xlim(cx - max_x_range / 2, cx + max_x_range / 2)
    axs[i].set_ylim(cy + max_y_range / 2, cy - max_y_range / 2)  # flip Y for top-down alignment

# Layout and colorbar
plt.tight_layout()

plt.savefig("/workdir/jp2626/chicken/scVelo/figures/Sham_LAL/noery_FB_netflow_plasticity.pdf", format='pdf', bbox_inches='tight', dpi=300, pad_inches=0)

plt.show()